# Récupération du classement journée par journée depuis J1

### On vient concat les journée 1 par 1 depuis J1 pour faire un CSV global

In [7]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from pprint import pprint

#### Aller chercher la journée maximum depuis le site du top 14

In [8]:
dico_id_club = {'LOU Rugby':1,
                'RC Toulon':2,
                'Section Paloise':3,
                'ASM Clermont':4,
                'Aviron Bayonnais':5,
                'Union Bordeaux-Bègles':6,
                'Castres Olympique':7,
                'Montpellier Hérault Rugby':8,
                'Stade Toulousain':9,
                'Racing 92':10,
                'Stade Rochelais':11,
                'CA Brive':12,
                'SU Agen':13,
                'Stade Français Paris':14,
                'Biarritz Olympique PB':15,
                'USA Perpignan':16,
                'Oyonnax Rugby':17,
                'RC Vannes':18}

In [9]:
df_global = pd.DataFrame()
list_annee = ['2019-2020','2020-2021', '2021-2022', '2022-2023', '2023-2024','2024-2025']

for annee in list_annee:
    for journee in range(1, 27):
        #print(f"{journee}, https://top14.lnr.fr/classement/{annee}/j{journee}?: Classement général") #try except sur chaque journée
        url=f"https://top14.lnr.fr/classement/{annee}/j{journee}?"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        journee_web_soup= soup.find_all('div', {'class': 'table-line table-line--ranking-full table-line--ranking-scrollable'})
        
           
        try:
            dico_urls = {f"https://top14.lnr.fr/classement/{annee}/j{journee}?": "Classement général"}
            for url in dico_urls.keys():
                dico_urls[url]
                # print(dico_urls[url])        
                # print(url)
                response = requests.get(url)
            
            
            test=journee_web_soup[0].find_all('div', {'class': ''})[1]
            
            journee_web=re.search(r'[0-9]+', str(test))[0]

            journee == int(journee_web)
        # print(journee_web)
            # print(response)
            # -------------------------------------------------------------------------------------
            
            
            if response.status_code == 200:
            
                # je crée une variable soup surn laquelle j'applique BeautifulSoup
                # avec en paramètre url.text et je découpe avec html.parser
                soup = BeautifulSoup(response.text, 'html.parser')
                # return(soup)   

                # NOM EQUIPES
                # je recherche les noms de toutes les équipes avec la balise a et la classe affichée sur la page internet
                list_nom_equipes = soup.find_all('a', {'class': 'base-link base-link--black'})

                # je crée une boucle insérer les noms des equipes dans une liste
                list_team = []
                for ele in list_nom_equipes:
                        ele = str(ele)
                        # Appliquer la regex à chaque élément `ele`
                        match = re.search(r'>([^<]+)<', ele)
                        if match:  # Vérifier si la regex a trouvé un résultat
                            list_team.append(match.group(1).strip())
                # print(list_team)    
                    
                points_equipes = soup.find_all('div', {'class': 'table-line__cell--small'})
                list_points_team = []
                for points in points_equipes:
                    # print(points)
                    points = str(points)
                    regex_points = re.search(r'[0-9]+', points)
                    # print(regex_points)
                    # regex_points[0]
                    # print(regex_points[0])
                    list_points_team.append(regex_points[0])

                all_team = soup.find_all('div', {'class': 'table-line table-line--ranking-full table-line--ranking-scrollable'})
                list_matchs_joues = []
                list_matchs_gagnes = []
                list_matchs_nuls = []
                list_matchs_perdus = []

                for team in range(0, 14):
                    for data in range(1, 5):
                        
                        if data == 1:
                            points_test = all_team[team].find_all('div', {'class': ''})[data]
                            # print(re.search(r'[0-9]+', str(points_test))[0])
                            list_matchs_joues.append(re.search(r'[0-9]+', str(points_test))[0])
                        elif data == 2:
                            points_test = all_team[team].find_all('div', {'class': ''})[data]
                            # print(re.search(r'[0-9]+', str(points_test))[0])
                            list_matchs_gagnes.append(re.search(r'[0-9]+', str(points_test))[0])
                        elif data == 3:
                            points_test = all_team[team].find_all('div', {'class': ''})[data]
                            # print(re.search(r'[0-9]+', str(points_test))[0])
                            list_matchs_nuls.append(re.search(r'[0-9]+', str(points_test))[0])
                        else:
                            points_test = all_team[team].find_all('div', {'class': ''})[data]
                            # print(re.search(r'[0-9]+', str(points_test))[0])
                            list_matchs_perdus.append(re.search(r'[0-9]+', str(points_test))[0])
                    
                            

                df = pd.DataFrame({
                        "Equipes": list_team,
                        "Pts": list_points_team,
                        "Matchs joués": list_matchs_joues,
                        "Gagnés": list_matchs_gagnes,
                        "Nuls": list_matchs_nuls,
                        "Perdus": list_matchs_perdus,
                        "Journée":f'J{journee}',
                        "Season": annee
                        })
                
            
                    
                    
                    #     # display(df)
                df_global = pd.concat([df_global, df])
                        

            
        except IndexError:
            break

KeyboardInterrupt: 

In [ ]:
df_global["ID_equipes"]= df_global["Equipes"].apply(lambda x : dico_id_club[x])

In [ ]:
df_global

,Equipes,Pts,Matchs joués,Gagnés,Nuls,Perdus,Journée,Season,ID_equipes
0,LOU Rugby,5,1,1,0,0,J1,2019-2020,1
1,RC Toulon,5,1,1,0,0,J1,2019-2020,2
2,Section Paloise,4,1,1,0,0,J1,2019-2020,3
3,ASM Clermont,4,1,1,0,0,J1,2019-2020,4
4,Aviron Bayonnais,4,1,1,0,0,J1,2019-2020,5
...,...,...,...,...,...,...,...,...,...
9,USA Perpignan,30,16,6,1,9,J26,2024-2025,16
10,Section Paloise,30,16,6,0,10,J26,2024-2025,3
11,Stade Français Paris,28,16,6,0,10,J26,2024-2025,14
12,Racing 92,27,16,5,1,10,J26,2024-2025,10


In [ ]:
df_global.to_csv('classement_jounees.csv')